# Notebook 5 - Implementation on Ukrainian Dataset
We used a Ukrainian trade dataset (`ukraine_trade_data_2018_ontology.csv`) as an input to our function to fully test our product.  

This dataset was similar in structure to our training dataset in that it had columns for `COMPANY_NAME`, `COMPANY_ID`, and `DESCRIPTION`.  However there were some elements of the dataset that needed to be fixed before processing. The column names were also different than those of our training dataset, but our product allows for column specification, so this was not an issue.

The results were positive!  Although a stronger metric is needed to identify similarity to known arms exporters, our product identified 32 INNs in the 7,326,528 row dataset as having similar text trading patterns as known Russian arms exporters.

In [1]:
#pip installations - necessary to get notebook to run
#update dask
!pip install --upgrade pip
!pip install dask==2.4.8
!pip install fsspec
!pip install --upgrade s3fs
!pip install numpy
!pip install pymystem3
!pip install spacy
!pip install joblib
!pip install pymorphy2==0.8
!pip install dask_ml

Requirement already up-to-date: pip in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (20.0.2)
ERROR: Could not find a version that satisfies the requirement dask==2.4.8 (from versions: 0.7.4.linux-x86_64, 0.7.5.linux-x86_64, 0.7.6.linux-x86_64, 0.8.1.macosx-10.5-x86_64, 0.2.0, 0.2.1, 0.2.2, 0.2.3, 0.2.4, 0.2.5, 0.2.6, 0.3.0, 0.4.0, 0.5.0, 0.6.0, 0.6.1, 0.7.0, 0.7.1, 0.7.2, 0.7.3, 0.7.4, 0.7.5, 0.7.6, 0.8.0, 0.8.1, 0.8.2, 0.9.0, 0.10.0, 0.10.1, 0.10.2, 0.11.0, 0.11.1, 0.12.0, 0.13.0rc1, 0.13.0, 0.14.0, 0.14.1, 0.14.2, 0.14.3, 0.15.0, 0.15.1, 0.15.2, 0.15.3, 0.15.4, 0.16.0, 0.16.1, 0.17.0, 0.17.1, 0.17.2, 0.17.3, 0.17.4, 0.17.5, 0.18.0, 0.18.1, 0.18.2, 0.19.0, 0.19.1, 0.19.2, 0.19.3, 0.19.4, 0.20.0, 0.20.1, 0.20.2, 1.0.0, 1.1.0, 1.1.1, 1.1.2, 1.1.3, 1.1.4, 1.1.5, 1.2.0, 1.2.1, 1.2.2, 2.0.0, 2.1.0, 2.2.0, 2.3.0, 2.4.0, 2.5.0, 2.5.2, 2.6.0, 2.7.0, 2.8.0, 2.8.1, 2.9.0, 2.9.1, 2.9.2, 2.10.0, 2.10.1, 2.11.0)
ERROR: No matching distribution found for dask==2.4.8
Requirement

In [2]:
# IMPORTS

# dataframe
import dask.dataframe as dd
import pandas as pd

# DESCRIPTION_GOOD preprocessing
import nltk
nltk.download("stopwords")
#--------#
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

# machine learning/analysis
from dask_ml.cluster import KMeans as dask_ml_model # sklearn's skmeans took up too much memory to run.

# measuring euclidian distance
from scipy.spatial.distance import euclidean, pdist

# S3 bucket interaction
import tempfile
import boto3
import joblib

# Disable warning message related to SettingWithCopyWarning
# displays when running final function otherwise
pd.options.mode.chained_assignment = None     # default = 'warn'

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/dask/array/random.py:27: FutureWarning: dask.array.random.doc_wraps is deprecated and will be removed in a future version
  FutureWarning,


In [3]:
# define stemmer and Russian stopwords for data preprocessing
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")
# https://stackoverflow.com/questions/5511708/adding-words-to-nltk-stoplist
# add trade-specific stopwords to list
newStopWords = ['г', '№', '10', '1', '20', '30', 'кг', '5', 'см',
                '100', '80', '2', 'х', 'l', 'м', '00', '000'
                '1.27', '2011.10631', '4', '12', '3', 'фр', 'количество',
                'становиться', 'мм', 'вид', 'упаковка', 'получать',
                'прочий', 'использование', 'масса', 'размер', 'черный',
                '6', '8', '7', '50', '40', '25', 'коробка', 'поддон',
                'вдоль', '250', '65', '85', '15', '35', '40', '45',
                '55', '60', '70', '75', 'м3', '13', '0', '14',
                '16', '18', 'm2', 'п', 'р', 'т', 'тип', 'являться',
                'размер', 'cm', 'm', '01', '02', '03', '04', '05',
                '06', '07', '08', '09', '24', '27']
russian_stopwords.extend(newStopWords)

#define function for preprocessing text - to be used later in notebook
#function will remove Russian stop words and any punctuation not removed in cleaning_trade_data_desc_kmeans.ipynb
def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
        and token != " " \
        and token.strip() not in punctuation]
    text = " ".join(tokens)
    return text

# similarity function for euclidian measure at end of main function
def similarity_func(u, v):
    return 1/(1+euclidean(u,v))

In [4]:
df = pd.read_csv('s3://ukraine-trade-data/ukraine_trade_data_2018_ontology.csv',dtype={'CONTRACTOR_ADDRESS': 'object', 
                                                                                       'EXCISE_DUTY_UAH': 'object'})

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (69) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df = df[['TRADE_DIRECTION', 'SHIPPER_NAME', 'SHIPPER_EDRPOU', 'DESCRIPTION_GOODS']]
df.columns

Index(['TRADE_DIRECTION', 'SHIPPER_NAME', 'SHIPPER_EDRPOU',
       'DESCRIPTION_GOODS'],
      dtype='object')

In [6]:
X = df.copy()

In [7]:
X.shape

(7326528, 4)

In [8]:
X = X[X['TRADE_DIRECTION'] == 'EXPORT']

In [9]:
X = X.drop(columns=['TRADE_DIRECTION'])

In [10]:
X = X.astype(str)

In [11]:
X.dtypes

SHIPPER_NAME         object
SHIPPER_EDRPOU       object
DESCRIPTION_GOODS    object
dtype: object

In [12]:
X = X.reset_index().drop(columns=['index'])

In [13]:
# all EDRPOU in ukrainian dataset were floats before we converted them to strings, have '.0' at end
# couldn't convert to int because of NaN values, didnt want to lose any data in dataset
# .str[:-2] removes last two characters from every string in column, in this case '.0'
X['SHIPPER_EDRPOU'] = X['SHIPPER_EDRPOU'].str[:-2]

In [14]:
X.head()

,SHIPPER_NAME,SHIPPER_EDRPOU,DESCRIPTION_GOODS
0,"ТОВАРИСТВО З ОБМЕЖЕНОЮ ВIДПОВIДАЛЬНIСТЮ ""ЕКО С...",40142870,1.МАКУХА СОЄВА НА КОРМА ДЛЯ ТВАРИН - 22000КГ
1,"ПРИВАТНЕ АКЦIОНЕРНЕ ТОВАРИСТВО ""ГРАФІЯ УКРАЇНА""",2469333,1.ВКЛАДИШІ ДО СИГАРЕТНОЇ ПАЧКИ: IC WI / IC WI ...
2,"ПРИВАТНЕ АКЦIОНЕРНЕ ТОВАРИСТВО ""НОВИЙ СТИЛЬ""",32565288,1.ГВИНТ З ЦИЛІНДРИЧНОЮ ГОЛ.ISO 4762-M8X2 0-8.8...
3,"ТОВАРИСТВО З ОБМЕЖЕНОЮ ВIДПОВIДАЛЬНIСТЮ ""ТАРТУ...",38282429,"1. ПЛОДИ СИРІ, ЗАМОРОЖЕНІ, БЕЗ ДОДАВАННЯ ЦУКРУ..."
4,"ПРИВАТНЕ АКЦIОНЕРНЕ ТОВАРИСТВО ""МАРІУПОЛЬСЬКИЙ...",191129,1.ТРУБА ЕЛЕКТРОЗВАРНА КВАДРАТНОГО ПЕРЕРІЗУ ЗГІ...


In [15]:
X.isna().count()

SHIPPER_NAME         1831495
SHIPPER_EDRPOU       1831495
DESCRIPTION_GOODS    1831495
dtype: int64

In [16]:
# # export ukraine to S3 bucket
# df.to_csv('s3://labs20-arms-bucket/data/ukraine.csv')

In [17]:
#load model, vectorizer, and tokenizer to notebook
s3 = boto3.resource('s3')
bucket=s3.Bucket('labs20-arms-bucket')

# load vectorizer from S3 bucket
key = "vectorizerf.pkl"
with tempfile.TemporaryFile() as fp:
    bucket.download_fileobj(Fileobj=fp, Key=key)
    fp.seek(0)
    vectorizer = joblib.load(fp)

# load model from S3 bucket
key = "modelf.pkl"
with tempfile.TemporaryFile() as fp:
    bucket.download_fileobj(Fileobj=fp, Key=key)
    fp.seek(0)
    model = joblib.load(fp)

#load cluster dataset from S3 bucket
# drop error column accidentally created in import
clusters = pd.read_csv('s3://labs20-arms-bucket/data/armsclustersf.csv')
clusters = clusters.drop([clusters.columns[0]], axis='columns')

# list of known arms exporters
inn_arms_exp_total = ['7718852163',  '7740000090',    '7731084175',  '6161021690',
                      '3807002509',  '6672315362',    '7802375335',  '7813132895',  
                      '7731280660',  '7303026762',    '5040007594',  '2501002394',  
                      '7807343496',  '7731559044',    '5042126251',  '7731595540',    
                      '7733018650',  '7722016820',    '7705654132',  '7714336520',    
                      '7801074335',  '6229031754',    '7830002462',  '6825000757',  
                      '5043000212',  '7802375889',    '5010031470',  '1660249187',  
                      '7720015691',  '6154573235',    '5038087144',  '7713006304',  
                      '7805326230',  '5023002050',    '4007017378',  '7714013456',  
                      '17718852163', '7811406004',    '7702077840',  '7839395419',  
                      '7702244226',  '7704721192',    '7731644035',  '7712040285',
                      '7811144648',  '4345047310',    '7720066255',  '6607000556',
                      '1832090230',  '1835011597',    '3305004083',  '4340000830',
                      '5074051432',  '1841015504',    '7105008338',  '7106002829', 
                      '7704274402',  '5942400228',    '7105514574',  '5012039795', 
                      '7714733528',  '3904065550',    '6825000757',  '7807343496', 
                      '7731559044',  '7805231691',    '7704859803',  '0273008320',
                      '7704274402',  '2902059091',    '7805034277',  '7727692011',
                      '7733759899',  '6154028021',    '7328032711',  '2635002815',
                      '5040097816',  '5027033274',    '5250018433',  '5200000046',
                      '7743813961',  '7718016666',    '5047118550',  '7704274402']

In [18]:
def process_predictor_function(df, name_column = 'CONSIGNOR_NAME', id_column = 'CONSIGNOR_INN', text_column = 'DESCRIPTION_GOOD',
                              invalid_id_terms = ['None', '00', 'ИНН/КПП НЕ О', '0'], min_trades=35, profile_similarity_threshold = .65,
                              cluster_columns = ['clust0', 'clust1', 'clust6']):
    """
    function to clean INNs of input dataframe and return Russian arms exporter similarity calculation
    
    """
    try:
        # set column variable
        # reduce dataframe so that dataframe only contains columns in columns variable
        df = df[[name_column, id_column, text_column]]
        
        # remove rows from dataset containing INNs of known arms exporters
        # check 'INN' column against inn_arms_exp_total list, drop row if there's a match with the list
        df = df[~df[id_column].isin(inn_arms_exp_total)]
        
        # clean INNs
        # Create subslice of dataframe for dictionary
        dict_df = df[[name_column, id_column]]
        # clean columns of dict_df, remove invalid_id_terms from CONSIGNOR_INN column
        invalid_id_terms = invalid_id_terms
        for term in invalid_id_terms:
            dict_df = dict_df[dict_df[id_column] != term]
        # drop all null values
        dict_df.dropna(inplace=True)
        # sort values by 'CONSIGNOR_NAME'
        dict_df.sort_values(name_column, inplace = True) 
        # dropping ALL duplicte 'CONSIGNOR_NAME' values from dictionary
        dict_df.drop_duplicates(subset =name_column, keep = 'first', inplace = True) 
        # create list of 2-item lists: [CONSIGNOR_NAME, CONSIGNOR_INN]
        new_list = dict_df.values.tolist()
        # create dictionary out of list of lists
        # for every list in the list of lists, take the first item in list (CONSIGNOR_NAME)
        # and add it to index position of dictionary, take second term ('CONSIGNOR_INN') and add it to value position of dictionary
        # cannot use pandas.to_dict() because it adds column names to dictionary; only want indexes/values
        new_dict = {t[0]:t[1] for t in new_list}
        # map new_dict to 'CONSIGNOR_INN' column of main dataframe
        df[id_column] = df[name_column].map(new_dict)

        # drop null values
        df.dropna(inplace=True)
        
        # remove all rows from list whose total INN count is less than min_trades variable
        # way to limit size before processing, weed out INNs that only have a few trades present in dataset
        df = df[df.groupby(id_column)[id_column].transform('size') >= min_trades]
        
        #create list for preprocessed text to be appended to
        processed_text_list = []
        
        #this is the alg to apply preprocessing function to text column
        # removed print statement from David's function
        for i in range(len(df[text_column])):
            x = df[text_column].iloc[i]
            if isinstance(x, str):
                processed_text_list.append(preprocess_text(x))
            else:
                processed_text_list.append(preprocess_text(x.astype(str)))
            
        # convert list of preprocessed text to dataframe
        # to be concatenated onto original dataframe
        df1 = pd.DataFrame({'PREPROCESSED_TEXT':processed_text_list})
        
        # reset indices of both dataframes
        df1 = df1.reset_index()
        df = df.reset_index()
        df['index'] = df.index
        
        # merge preprocessed text to original dataframe
        df_merge = pd.concat([df, df1], axis=1, join='inner')
        
        # drop DESCRIPTION_GOOD column, no longer necessary now that PROCESSED_TEXT column is present
        df_merge = df_merge.drop([text_column, 'index'], axis='columns')
        
        #define variable to feed to TFIDF Vectorizer - 'PROCESSED_TEXT' column of train dataset
        text = df_merge['PREPROCESSED_TEXT']
        
        #transform text with vectorizer
        #Converted to Unicode because it will run into an np.nan error. This need to be turned into a unicode string.
        sparse = vectorizer.transform(text.values.astype('U'))
        
        # Get feature names to use as dataframe column headers
        dtm = pd.DataFrame(sparse.todense(), columns=vectorizer.get_feature_names())
        
        # reset indices of both dataframes for merge
        # not sure why we had to do this, but running the following three commands gave us the results we wanted
        dtm = dtm.reset_index()
        df_merge = df_merge.reset_index()
        df_merge['index'] = df_merge.index
        dtm['index'] = dtm.index
        
        # merge vectorized word feature matrix with training dataset
        df_merge_vector = pd.concat([df_merge, dtm], axis=1, join='inner')
        # drop index columns
        df_merge_vector = df_merge_vector.drop(columns=['index'])
        
        # variable manipulation to feed into KMeans model
        # pull create variable containing dataframe of vectorized words only, all rows, columns indexed 4 and onward
        X = df_merge_vector.drop(columns=[name_column, id_column, 'PREPROCESSED_TEXT'])
        
        # convert X dataframe into array
        # necessary to feed to KMeans model
        X_array = X.values
        
        # fit model on vectorized word array
        labels = model.predict(X_array)
        
        # create 'cluster' column to add to vectorized dataframe
        #Glue back to originaal data
        df_merge_vector['cluster'] = labels

        # extract columns for final analysis
        Y = df_merge_vector[[id_column,'cluster']]
        
        # add column to dataframe for each cluster in model, created with copied values from 'cluster' column
        # create 1,0 boolean to check if number in cell is equal to number of cluster, assigns 1s and 0s accordingly
        # drop cluster column, no longer necessary now that we have count
        for i in range(model.n_clusters):
            Y['clust{}'.format(i)] = Y['cluster']
            Y['clust{}'.format(i)] = (Y['clust{}'.format(i)] == i) * 1
        
        # drop 'cluster' column, no longer necessary now that we have total trades per cluster per INN
        Y = Y.drop(columns=['cluster'])
        
        #create column_names variable to filter out CONSIGNER_INN from .groupby() in next step
        column_names = Y.drop(columns = [id_column]).columns.tolist()
        
        #create new dataframe totalling trades per cluster per INN
        Y = pd.DataFrame(Y.groupby([Y[id_column]])[column_names].sum()).reset_index()
        
        # add final tally for known arms exporters
        # reset index so known arms exporters are at bottom of dataframe, indexed properly
        Y = Y.append(clusters.iloc[0,1:], sort=None).reset_index().drop(columns=['index'])
        
        # convert all columns except for 'CONSIGNOR_INN' to decimals/percentages of total
        Y[column_names] = Y[column_names].div(Y[column_names].sum(axis=1), axis=0)
        
        # cluster columns
        # remove clusters with low percentages for known arms exporters from dataset
        cluster_columns = cluster_columns
        cluster_columns.insert(0, id_column)
        Y = Y[cluster_columns]
        
        # similarity matrix - create list of p-distance scores using pdistance & euclidian distance
        # simply put, it measures how similar two sets if numbers are
        # https://stackoverflow.com/questions/35758612/most-efficient-way-to-construct-similarity-matrix
        # each row in dataframe will be compared against the bottom row of the dataframe, which contains the totals for knowns arms exporters
        pscores=[]
        for i in range(len(Y)):
            x = pdist([Y.iloc[-1, 1:],Y.iloc[i, 1:]], similarity_func)[0]
            pscores.append(x)
        
        # add pdist_score column to Y dataframe
        Y['pdist_score'] = pscores
        
        # drop control row (known arms exporters totals)
        Y = Y.drop(Y.index[-1])
        
        # create profile_similarity_threshold variable
        # if INN's pdist_score >= profile_similarity_threshold, INN will be included in final dataframe
        # if INN's pdist_score < profile_similarity_threshold, INN will not be included in final dataframe
        Y = Y[Y['pdist_score'] >= profile_similarity_threshold]
        
        #generate dataframe
        return Y
        
    except:
        
        raise

In [21]:
# test = process_predictor_function(X, name_column = 'SHIPPER_NAME', id_column = 'SHIPPER_EDRPOU', text_column = 'DESCRIPTION_GOODS',
#                               profile_similarity_threshold = .85)
test.sort_values(by='pdist_score', ascending=True).tail(25)

,SHIPPER_EDRPOU,clust0,clust1,clust6,pdist_score
940,2781504255,0.837838,0.162162,0.0,0.860791
1834,32502825,0.759494,0.240506,0.0,0.860918
1497,31489175,0.761194,0.238806,0.0,0.861515
2840,36767366,0.833333,0.166667,0.0,0.862335
2755,36417791,0.837989,0.117318,0.0,0.862404
3468,38884736,0.766667,0.233333,0.0,0.863279
3298,382272,0.829268,0.170732,0.0,0.863586
3774,39792657,0.826923,0.173077,0.0,0.864244
697,24363204,0.770950,0.229050,0.0,0.864483
1587,31804036,0.773869,0.226131,0.0,0.865212


In [20]:
test.shape

(65, 5)